### Cosc2671 Assignment 2 Group 15 


In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from nltk.sentiment import SentimentIntensityAnalyzer
import seaborn as sns
import nltk
import praw
import datetime as dt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Reddit API credentials
clientid = "FgY4iejqu83Q9QpgNMX9dQ"
clientsecret = "eeSZYZUjfMIAQ5qcULOc9SSYOHghFQ"
password = "1357900jkx"
username = "Traditional-Shine402"
useragents = 'client for SNAM2024'
reddit = praw.Reddit(client_id=clientid, client_secret=clientsecret,password=password,
                           username=username,user_agent=useragents)


# Keywords and Subreddits
keywords = [
    "moon landing fake", 
    "moon landing hoax", 
    "fake moon landing", 
    "did we land on the moon", 
    "moon landing conspiracy", 
    "moon landing was staged", 
    "apollo mission fake", 
    "moon landing faked", 
    "moon hoax", 
    "apollo hoax", 
    "moon conspiracy", 
    "moon landing lies", 
    "nasa faked moon landing"]

subreddits = ["conspiracy", "moonlandingfake"]


In [3]:
#Data collection
posts = []

for subreddit in subreddits:
    for keyword in keywords:
        for submission in reddit.subreddit(subreddit).search(keyword, limit=100):
            posts.append({
                'subreddit': subreddit,
                'title': submission.title,
                'text': submission.selftext,
                'author': str(submission.author),
                'score': submission.score,
                'created_utc': dt.datetime.fromtimestamp(submission.created_utc),
                'id': submission.id,
                'num_comments': submission.num_comments
            })

posts_df = pd.DataFrame(posts)
posts_df.to_csv("reddit_posts.csv", index=False)
print(f"Fetched {len(posts_df)} posts.")

# Fetch comments
comments_data = []

for post_id in posts_df['id']:
    if len(comments_data) > 25000:
        break
    submission = reddit.submission(id=post_id)
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        comments_data.append({
            'post_id': post_id,
            'comment_body': comment.body,
            'author': str(comment.author),
            'score': comment.score,
            'created_utc': dt.datetime.fromtimestamp(comment.created_utc)
        })

comments_df = pd.DataFrame(comments_data)
comments_df.to_csv("reddit_comments.csv", index=False)
print(f"Fetched {len(comments_df)} comments.")

Fetched 1474 posts.
Fetched 25005 comments.


In [4]:
posts_df

,subreddit,title,text,author,score,created_utc,id,num_comments
0,conspiracy,Did we ever land on the moon…?,,Own_Teacher3433,739,2024-08-23 22:28:45,1ezbh2v,759
1,conspiracy,Can you tell me why you think the moon landing...,I come in peace! I am a about to start my PhD ...,Ms_Photon,866,2023-03-12 01:48:43,11olzpd,1374
2,conspiracy,"If the moon landing was fake, why the enemies ...",,TimmyOTule,196,2024-09-27 22:26:50,1fqmhry,527
3,conspiracy,Why do people get so defensive when the 1969 M...,Whenever a Moon Landing conspiracy is posted t...,Majin_Vegeta_,251,2024-01-03 10:27:29,18x3uai,449
4,conspiracy,If the moon landing was faked how have other c...,"In my limited understanding of space travel, w...",5afari,388,2022-07-13 22:57:31,vy2lws,525
...,...,...,...,...,...,...,...,...
1469,moonlandingfake,Why I no Longer Believe the Apollo Moon Landin...,,HelicopterJesus,3,2021-10-27 04:31:21,qgbkci,1
1470,moonlandingfake,Here's the truth,Moon landings were faked on the Moon.💀,Sea-Mark-7504,11,2023-03-04 04:55:36,11h8s9e,5
1471,moonlandingfake,Bruh,I've gotten 50 new types of brain cancer from ...,YesterdayDirect8401,19,2022-11-26 00:36:44,z4dkp6,6
1472,moonlandingfake,Is it me or has the internet become the most e...,I used to use it to win all kinds of arguments...,ElectronicPresenc,11,2022-09-21 16:45:45,xjxct3,7


In [5]:
comments_df

,post_id,comment_body,author,score,created_utc
0,1ezbh2v,###[Meta] Sticky Comment\n\n[Rule 2](https://w...,AutoModerator,1,2024-08-23 22:28:47
1,1ezbh2v,I once asked A.I how to get sugar out of a gas...,dinahmoon,803,2024-08-24 01:15:41
2,1ezbh2v,Everyone loves AI when it gives you the answer...,Binarydemons,157,2024-08-24 03:49:59
3,1ezbh2v,Wow that AI has such an unbiased sounding name,before686entenz,644,2024-08-23 23:18:37
4,1ezbh2v,"I don't know, but China confirmed their probes...",AccordingWarning9534,323,2024-08-24 01:16:11
...,...,...,...,...,...
25000,1dhq6a5,Ahh yes -- more bold claims without **ANY** de...,dcforce,0,2024-06-18 04:20:50
25001,1dhq6a5,I understand maybe moon landing being fake but...,Ok_Land_832,1,2024-06-18 16:40:33
25002,1dhq6a5,"Not given by NASA, not claimed to be real by N...",4544BeersOnTheWall,3,2024-06-18 07:41:37
25003,1dhq6a5,Have you even read this article?,Darkherring1,1,2024-06-18 13:02:47
